In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
#loading master_data2.csv
data2 = pd.read_csv('saved_datasets/master_data2.csv')

In [12]:
#total unique user_id
data2['user_id'].nunique()

2192

In [13]:
#reordered count/total rows : reordered = 1 percentage
data2[data2['reordered'] == 1].shape[0]/data2.shape[0]

0.5835062884714314

In [14]:
# Spliting the data train validation and test using stratified sampling on reordered
from sklearn.model_selection import train_test_split
train, test = train_test_split(data2, test_size=0.2, stratify=data2['reordered'])
train, val = train_test_split(train, test_size=0.2, stratify=train['reordered'])

train.shape, val.shape, test.shape


((216472, 15), (54118, 15), (67648, 15))

In [15]:
# percentage of reordered = 1 in train, val and test
train[train['reordered'] == 1].shape[0]/train.shape[0], val[val['reordered'] == 1].shape[0]/val.shape[0], test[test['reordered'] == 1].shape[0]/test.shape[0]

(0.5835073358217229, 0.5835027162866329, 0.5835057947019867)

In [16]:
train.head()

,order_id,product_id,add_to_cart_order,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department,reordered
48989,502709,17579,3,732,prior,35,6,12,8.0,Cream Top Strawberry on the Bottom Yogurt,120,16,yogurt,dairy eggs,1
82334,861981,14803,6,1759,prior,39,3,14,1.0,Mild White Kimchi Napa Cabbage,110,13,pickled goods olives,pantry,1
317545,3351274,25804,2,694,prior,4,5,6,30.0,Original Soy Milk,91,16,soy lactosefree,dairy eggs,1
270015,2848647,15290,45,1501,prior,8,2,11,8.0,Orange Bell Pepper,83,4,fresh vegetables,produce,0
4273,45068,3880,6,1845,prior,4,1,14,30.0,Corn Maize Tortillas,128,3,tortillas flat bread,bakery,1


In [17]:
#function to drop order_id, eval_set, product_id, aisle_id and department_id
def drop_columns(data):
    data = data.drop(['order_id', 'eval_set', 'product_id', 'aisle_id', 'department_id'], axis=1)
    return data

#function to seperate features and target
def feature_target(data):
    X = data.drop('reordered', axis=1)
    y = data['reordered']
    return X, y



In [18]:
#dropping columns
train = drop_columns(train)
train.head()



,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle,department,reordered
48989,3,732,35,6,12,8.0,Cream Top Strawberry on the Bottom Yogurt,yogurt,dairy eggs,1
82334,6,1759,39,3,14,1.0,Mild White Kimchi Napa Cabbage,pickled goods olives,pantry,1
317545,2,694,4,5,6,30.0,Original Soy Milk,soy lactosefree,dairy eggs,1
270015,45,1501,8,2,11,8.0,Orange Bell Pepper,fresh vegetables,produce,0
4273,6,1845,4,1,14,30.0,Corn Maize Tortillas,tortillas flat bread,bakery,1


In [19]:
#adding a feature: average days_since_prior_order for each user_id

def avg_days_since_prior_order(data):
    days_since_prior_order = data.groupby('user_id')['days_since_prior_order'].mean().reset_index()
    data = data.merge(days_since_prior_order, on='user_id', how='left')
    data = data.rename(columns={'days_since_prior_order_x':'days_since_prior_order', 'days_since_prior_order_y':'avg_days_since_prior_order'})
    return data

train = avg_days_since_prior_order(train)
train.head()

,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle,department,reordered,avg_days_since_prior_order
0,3,732,35,6,12,8.0,Cream Top Strawberry on the Bottom Yogurt,yogurt,dairy eggs,1,8.787879
1,6,1759,39,3,14,1.0,Mild White Kimchi Napa Cabbage,pickled goods olives,pantry,1,6.303665
2,2,694,4,5,6,30.0,Original Soy Milk,soy lactosefree,dairy eggs,1,22.617647
3,45,1501,8,2,11,8.0,Orange Bell Pepper,fresh vegetables,produce,0,13.606299
4,6,1845,4,1,14,30.0,Corn Maize Tortillas,tortillas flat bread,bakery,1,28.297297


In [20]:
#most_common order_dow for each user_id : mode of order_dow for each user_id
def most_common_order_dow(data):
    order_dow = data.groupby('user_id')['order_dow'].agg(lambda x:x.value_counts().index[0]).reset_index()
    data = data.merge(order_dow, on='user_id', how='left')
    data = data.rename(columns={'order_dow_x':'order_dow', 'order_dow_y':'most_common_order_dow'})
    return data

train = most_common_order_dow(train)
train.head()

,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle,department,reordered,avg_days_since_prior_order,most_common_order_dow
0,3,732,35,6,12,8.0,Cream Top Strawberry on the Bottom Yogurt,yogurt,dairy eggs,1,8.787879,3
1,6,1759,39,3,14,1.0,Mild White Kimchi Napa Cabbage,pickled goods olives,pantry,1,6.303665,5
2,2,694,4,5,6,30.0,Original Soy Milk,soy lactosefree,dairy eggs,1,22.617647,6
3,45,1501,8,2,11,8.0,Orange Bell Pepper,fresh vegetables,produce,0,13.606299,2
4,6,1845,4,1,14,30.0,Corn Maize Tortillas,tortillas flat bread,bakery,1,28.297297,1


In [21]:
# check unique values of order_hour_of_day
train['order_hour_of_day'].unique()

array([12, 14,  6, 11, 17, 15, 18, 13,  9, 10, 22, 16, 20,  3,  7,  8, 23,
        0,  2, 21, 19,  1,  4,  5])

In [22]:
#spliting order_hour_of_day into 4 categories
def split_order_hour_of_day(data):
    data['order_hour_of_day'] = pd.cut(data['order_hour_of_day'], bins=[0, 6, 12, 18, 24], labels=['night', 'morning', 'afternoon', 'evening'])
    return data

train = split_order_hour_of_day(train)
train.head()


,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle,department,reordered,avg_days_since_prior_order,most_common_order_dow
0,3,732,35,6,morning,8.0,Cream Top Strawberry on the Bottom Yogurt,yogurt,dairy eggs,1,8.787879,3
1,6,1759,39,3,afternoon,1.0,Mild White Kimchi Napa Cabbage,pickled goods olives,pantry,1,6.303665,5
2,2,694,4,5,night,30.0,Original Soy Milk,soy lactosefree,dairy eggs,1,22.617647,6
3,45,1501,8,2,morning,8.0,Orange Bell Pepper,fresh vegetables,produce,0,13.606299,2
4,6,1845,4,1,afternoon,30.0,Corn Maize Tortillas,tortillas flat bread,bakery,1,28.297297,1


In [23]:
#total number of orders for each user_id
def total_orders(data):
    total_orders = data.groupby('user_id')['order_number'].max().reset_index()
    data = data.merge(total_orders, on='user_id', how='left')
    data = data.rename(columns={'order_number_x':'order_number', 'order_number_y':'total_orders'})
    return data

train = total_orders(train)
train.head()

,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle,department,reordered,avg_days_since_prior_order,most_common_order_dow,total_orders
0,3,732,35,6,morning,8.0,Cream Top Strawberry on the Bottom Yogurt,yogurt,dairy eggs,1,8.787879,3,40
1,6,1759,39,3,afternoon,1.0,Mild White Kimchi Napa Cabbage,pickled goods olives,pantry,1,6.303665,5,61
2,2,694,4,5,night,30.0,Original Soy Milk,soy lactosefree,dairy eggs,1,22.617647,6,9
3,45,1501,8,2,morning,8.0,Orange Bell Pepper,fresh vegetables,produce,0,13.606299,2,10
4,6,1845,4,1,afternoon,30.0,Corn Maize Tortillas,tortillas flat bread,bakery,1,28.297297,1,6


In [24]:
#average number of products in each order for each user_id
def avg_products(data):
    avg_products = data.groupby('user_id')['add_to_cart_order'].mean().reset_index()
    data = data.merge(avg_products, on='user_id', how='left')
    data = data.rename(columns={'add_to_cart_order_x':'add_to_cart_order', 'add_to_cart_order_y':'avg_products'})
    return data

train = avg_products(train)
train.head()

,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle,department,reordered,avg_days_since_prior_order,most_common_order_dow,total_orders,avg_products
0,3,732,35,6,morning,8.0,Cream Top Strawberry on the Bottom Yogurt,yogurt,dairy eggs,1,8.787879,3,40,3.398058
1,6,1759,39,3,afternoon,1.0,Mild White Kimchi Napa Cabbage,pickled goods olives,pantry,1,6.303665,5,61,5.816062
2,2,694,4,5,night,30.0,Original Soy Milk,soy lactosefree,dairy eggs,1,22.617647,6,9,4.743590
3,45,1501,8,2,morning,8.0,Orange Bell Pepper,fresh vegetables,produce,0,13.606299,2,10,16.632353
4,6,1845,4,1,afternoon,30.0,Corn Maize Tortillas,tortillas flat bread,bakery,1,28.297297,1,6,6.553191


In [25]:
# check for nan values
train.isnull().sum()


add_to_cart_order                 0
user_id                           0
order_number                      0
order_dow                         0
order_hour_of_day              1176
days_since_prior_order        13689
product_name                      0
aisle                             0
department                        0
reordered                         0
avg_days_since_prior_order        0
most_common_order_dow             0
total_orders                      0
avg_products                      0
dtype: int64

In [26]:
#show rows where days_since_prior_order is nan
train[train['days_since_prior_order'].isnull()]
#imputing nan values with 0 as this is the first order
train['days_since_prior_order'] = train['days_since_prior_order'].fillna(0)

In [27]:
#imputing order_hour_of_day with most common order_hour_of_day
train['order_hour_of_day'] = train['order_hour_of_day'].fillna(train['order_hour_of_day'].mode()[0])

In [28]:
#seperating features and target
X_train, y_train = feature_target(train)

In [30]:
categorical_columns = ['order_dow', 'order_hour_of_day','product_name', 'aisle', 'department', 'most_common_order_dow']

#count of unique values in each categorical column
for column in categorical_columns:
    print(column, X_train[column].nunique())

order_dow 7
order_hour_of_day 4
product_name 18836
aisle 134
department 21
most_common_order_dow 7


In [31]:
#product_name has too many columns so apply ordinal encoding
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
X_train['product_name'] = encoder.fit_transform(X_train['product_name'].values.reshape(-1,1))

#one hot encoding for other categorical columns
X_train = pd.get_dummies(X_train, columns=['order_dow', 'order_hour_of_day', 'aisle', 'department', 'most_common_order_dow'])

X_train.head()

,add_to_cart_order,user_id,order_number,days_since_prior_order,product_name,avg_days_since_prior_order,total_orders,avg_products,order_dow_0,order_dow_1,...,department_pets,department_produce,department_snacks,most_common_order_dow_0,most_common_order_dow_1,most_common_order_dow_2,most_common_order_dow_3,most_common_order_dow_4,most_common_order_dow_5,most_common_order_dow_6
0,3,732,35,8.0,4044.0,8.787879,40,3.398058,0,0,...,0,0,0,0,0,0,1,0,0,0
1,6,1759,39,1.0,9245.0,6.303665,61,5.816062,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,694,4,30.0,12916.0,22.617647,9,4.743590,0,0,...,0,0,0,0,0,0,0,0,0,1
3,45,1501,8,8.0,10300.0,13.606299,10,16.632353,0,0,...,0,1,0,0,0,1,0,0,0,0
4,6,1845,4,30.0,3884.0,28.297297,6,6.553191,0,1,...,0,0,0,0,1,0,0,0,0,0
